# Big Data Final Project
The aim of this project is to analyze the chat messages that flow into a popular Battlefield 4 server. The dataset is quite large (84MB) at the time of collection.

I'm going to do a simple wordcount, as well as some specific cleanups that are required to get this data useable, such as removing messages that are server-generated, and some messages that have less than desireable words.

The complete dataset will not be available long after publishing for privacy reasons.